In [1]:
# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [2]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [3]:
# os.environ['WANDB_NOTEBOOK_NAME'] = 'RUN_1'
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lihem (takim). Use `wandb login --relogin` to force relogin


True

In [4]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'normalization': {
          'values': [False, True]
        }
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100},
    'learning_rate': {
        'value': 0.000063
        },
    'batch_size': {
          'value': 64
        },
    'dropout': {
          'value': True
        },
    'batchnorm': {
          'value': True
        },
    'regularization': {
          'value': False
        },
    })

In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'value': 64},
                'batchnorm': {'value': True},
                'dropout': {'value': True},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'learning_rate': {'value': 6.3e-05},
                'normalization': {'values': [False, True]},
                'regularization': {'value': False}}}


In [6]:
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: 0k41xr03
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/0k41xr03


In [7]:
import tensorflow as tf

# Define the AlexNet architecture
def create_model(dropout, batchnorm, regularization):

    model = tf.keras.Sequential()

    if regularization:
        model.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(2, 2), activation='relu', input_shape=(32, 32, 3), kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    else:
        model.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(2, 2), activation='relu', input_shape=(32, 32, 3)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))


    if regularization:
        model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    else:
        model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))


    if regularization:
        model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
        model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
        model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    else:
        model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
        model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
        model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))


    model.add(tf.keras.layers.Flatten())


    model.add(tf.keras.layers.Dense(units=4096, activation='relu'))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))


    model.add(tf.keras.layers.Dense(units=4096, activation='relu'))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))


    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    return model

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        x_train_to_use = (x_train.astype('float32') / 255) if config['normalization'] else x_train
        x_test_to_use = (x_test.astype('float32') / 255) if config['normalization'] else x_test

        tf.keras.backend.clear_session()
        model = create_model(config["dropout"], config["batchnorm"], config["regularization"])
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        history = model.fit(x_train_to_use, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )
        
        test_stats = model.evaluate(x_test_to_use, y_test)
        wandb.log({"test_loss": test_stats[0]})
        wandb.log({"test_acc": test_stats[1]})

        val_loss_history = history.history['val_loss']
        val_acc_history = history.history['val_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [9]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: teffd0qn with config:
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 6.3e-05
wandb: 	normalization: False
wandb: 	regularization: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 19s 23ms/step - loss: 1.9588 - accuracy: 0.3630 - top@3_accuracy: 0.7016 - val_loss: 1.5109 - val_accuracy: 0.4546 - val_top@3_accuracy: 0.7924
Epoch 2/100
704/704 [==============================] - 15s 22ms/step - loss: 1.4450 - accuracy: 0.4886 - top@3_accuracy: 0.8099 - val_loss: 1.3964 - val_accuracy: 0.4912 - val_top@3_accuracy: 0.8194
Epoch 3/100
704/704 [==============================] - 15s 22ms/step - loss: 1.2895 - accuracy: 0.5439 - top@3_accuracy: 0.8394 - val_loss: 1.5459 - val_accuracy: 0.4430 - val_top@3_accuracy: 0.7764
Epoch 4/100
704/704 [==============================] - 16s 22ms/step - loss: 1.1735 - accuracy: 0.5851 - top@3_accuracy: 0.8634 - val_loss: 1.2737 - val_accuracy: 0.5454 - val_top@3_accuracy: 0.8510
Epoch 5/100
704/704 [==============================] - 16s 22ms/step - loss: 1.0767 - accuracy: 0.6236 - top@3_accuracy: 0.8836 - val_loss: 1.2344 - val_accuracy: 0.5540 - val_top@3_accuracy: 0.8662
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▃▃▄▄▅▅▅▆▆▆▆▇▇▇▇█████
epoch/epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
epoch/top@3_accuracy,▁▄▄▅▅▆▆▆▇▇▇▇▇████████
epoch/val_accuracy,▁▂▁▄▄▁▆▅▅▆▇▇▆▇▇▇█▇▇▇▆
epoch/val_loss,▆▅▇▄▃▇▂▃▃▃▁▁▃▂▃▃▂▄▄▄█
epoch/val_top@3_accuracy,▃▄▂▆▆▁▇▆▅▆▇▇▇▇▇▇█▇▇▇▅
test_acc,▁


wandb: Agent Starting Run: grj5k8k8 with config:
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 6.3e-05
wandb: 	normalization: True
wandb: 	regularization: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 17s 23ms/step - loss: 1.8187 - accuracy: 0.3835 - top@3_accuracy: 0.7193 - val_loss: 2.0081 - val_accuracy: 0.3136 - val_top@3_accuracy: 0.6922
Epoch 2/100
704/704 [==============================] - 15s 22ms/step - loss: 1.3942 - accuracy: 0.5028 - top@3_accuracy: 0.8170 - val_loss: 1.5525 - val_accuracy: 0.4606 - val_top@3_accuracy: 0.7730
Epoch 3/100
704/704 [==============================] - 15s 22ms/step - loss: 1.2362 - accuracy: 0.5637 - top@3_accuracy: 0.8519 - val_loss: 1.2886 - val_accuracy: 0.5542 - val_top@3_accuracy: 0.8352
Epoch 4/100
704/704 [==============================] - 15s 22ms/step - loss: 1.1467 - accuracy: 0.5956 - top@3_accuracy: 0.8705 - val_loss: 1.2107 - val_accuracy: 0.5766 - val_top@3_accuracy: 0.8608
Epoch 5/100
704/704 [==============================] - 15s 22ms/step - loss: 1.0434 - accuracy: 0.6348 - top@3_accuracy: 0.8877 - val_loss: 1.1821 - val_accuracy: 0.5882 - val_top@3_accuracy: 0.8628
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▂▃▄▄▄▅▅▅▆▆▆▇▇▇▇▇██████
epoch/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▃▄▅▅▆▆▆▇▇▇▇▇██████████
epoch/val_accuracy,▁▄▅▆▆▆▆▆▇▆▇▇▇▇▇▇████▇▇▇
epoch/val_loss,█▅▃▂▂▂▂▂▂▃▂▂▁▃▃▂▂▂▃▂▃▄▅
epoch/val_top@3_accuracy,▁▄▆▆▇▆▇▆▇▆▇▇█▇▇███▇██▇▇
test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
